# This version uses the same logic as in v_10 but uses one week as its timeframe

#### Have also updated the FCR and aFRR functions to make them faster and make FCR handle missing values

In [15]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, analysis, timeframes, data_handling
import numpy as np
import pickle
from datetime import datetime



In [2]:
timeframe = timeframes.two_days
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)

In [3]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [4]:

markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())
price_list = [market.price_data for market in M]
volume_list = [market.volume_data for market in M]


In [5]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2036
Amount of meters with direction down or both : 2051
Amount of hours : 48


In [6]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
total_response_time = np.sum(R_h_l) # total response time
#total_flex = total_up_flex + total_down_flex
average_response_time = total_response_time/ (len(H)*len(L))
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex} MW")
print(f"Total down flex volume: {total_down_flex} MW")
print(f"Average flex volume pr hour up: {hourly_flex_up} MWh")
print(f"Average flex volume pr hour down: {hourly_flex_down} MWh")
print(f"Average response time: {average_response_time} seconds")

Total up flex volume: 300.572342 MW
Total down flex volume: 297.23245299999996 MW
Average flex volume pr hour up: 6.261923791666667 MWh
Average flex volume pr hour down: 6.192342770833332 MWh
Average response time: 146.42320627190603 seconds


In [7]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]

In [8]:
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H = H, M = M, L = L, freq_data= freq_data, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, P_h_m = P_h_m, Vp_h_m = Vp_h_m, F = F, markets_dict= markets_name_dict, timeframe = timeframe)


/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [9]:
compatible_list = utils.get_compatibility_dict(L, M)

In [10]:
import math
def get_batched_versions(L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], H : [pd.Timestamp], F : dict, freq_data :pd.DataFrame, P_h_m : np.array, Vp_h_m : np.array, R_h_l : np.array, Fu_h_l : np.array, Fd_h_l : np.array):
    """ Function to only fetch the collections that are batched in the batched optimization model. Can use this to check if there is something wrong with some of the batches.

    Args:
        L (list(new_meters.PowerMeter]): set of all meters
        M (list(final_markets.ReserveMarket]): set of all markets
        H (list(pd.Timestamp]): set of all hours
        F (dict): Dictionary to find the activation percentages for each market and hour
        freq_data (pd.DataFrame): dataframe with the frequency data
        P_h_m (np.array): The price for each hour and market
        Vp_h_m (np.array): The volume for each hour and market
        R_h_l (np.array): Response time for each load each hour
        Fu_h_l (np.array): Up flex volume for each load that are compatible with up markets for each hour 
        Fd_h_l (np.array): Down flex volume for each load that are compatible with down markets for each hour
       

    Returns:
        batched_results (dict): Dictionary with the batched results
    """
    batch_size = 24  # For example, batching by 24 hours
    num_batches = math.ceil(len(H) / batch_size)
    aggregated_results = {
        'batched H': [],
        'batched R_h_l': [],
        'batched Fu_h_l': [],
        'batched Fd_h_l': [],
        'batched Vp_h_m': [],
        'batched P_h_m': [],
        'batched timeframes': [],
        'batched Ir_h_l_m': [],
        'batched Ia_h_l_m': [],
        'batched Va_hm': []
    }
    market_name_dict = {m.name : m for m in M}

    for b in range(num_batches):
        # Determine the subset of hours for this batch
        start_index = b * batch_size
        end_index = min((b + 1) * batch_size, len(H))
        batch_H = H[start_index:end_index]

        # Slice numpy arrays for the current batch
        batch_R_h_l = R_h_l[start_index:end_index, :]
        batch_Fu_h_l = Fu_h_l[start_index:end_index, :]
        batch_Fd_h_l = Fd_h_l[start_index:end_index, :]
        batch_Vp_h_m = Vp_h_m[start_index:end_index, :]
        batch_P_h_m = P_h_m[start_index:end_index, :]
        tf = timeframes.TimeFrame(year = 2023, start_month = 6, end_month = 6, start_day = batch_H[0].day, end_day = batch_H[0].day, start_hour = 0, end_hour = 23)

        # the income
        batch_Ir_hlm, batch_Ia_hlm, batch_Va_hm = utils.get_income_dictionaries(H = batch_H, M = M, L =L, freq_data = freq_data, Fu_h_l = batch_Fu_h_l, Fd_h_l = batch_Fd_h_l, P_h_m = batch_P_h_m, Vp_h_m = batch_Vp_h_m, F =F, markets_dict = market_name_dict, timeframe = tf)
       

        # Run the optimization model for this batch
        #_, x, y, w, d = run_optimization_model(L= L, M= M, H = batch_H,F= F, Ir_hlm= batch_Ir_hlm, Ia_hlm= batch_Ia_hlm, Va_hm= batch_Va_hm, Vp_h_m= batch_Vp_h_m, Vm_m=Vm_m, R_m=R_m, R_h_l=batch_R_h_l, Fu_h_l=batch_Fu_h_l, Fd_h_l=batch_Fd_h_l, compatible_list=compatible_list, log_filename=log_filename, model_name=f"{model_name}_batch_{b}")
        # Store results
        #aggregated_results['models'].append(model)
        aggregated_results['batched H'].append(batch_H)
        aggregated_results['batched R_h_l'].append(batch_R_h_l)
        aggregated_results['batched Fu_h_l'].append(batch_Fu_h_l)
        aggregated_results['batched Fd_h_l'].append(batch_Fd_h_l)
        aggregated_results['batched Vp_h_m'].append(batch_Vp_h_m)
        aggregated_results['batched P_h_m'].append(batch_P_h_m)
        aggregated_results['batched timeframes'].append(tf)
        aggregated_results['batched Ir_h_l_m'].append(batch_Ir_hlm)
        aggregated_results['batched Ia_h_l_m'].append(batch_Ia_hlm)
        aggregated_results['batched Va_hm'].append(batch_Va_hm)
        


    # Process aggregated_results as needed
    return aggregated_results

In [11]:
batch_vals = get_batched_versions(L= L, M= M, H = H, F = F,freq_data=freq_data, P_h_m=P_h_m , Vp_h_m =Vp_h_m, R_h_l = R_h_l, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l)

/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [13]:
#for day in range(len(batch_vals["batched H"])):

"""
'batched H': [],
'batched R_h_l': [],
'batched Fu_h_l': [],
'batched Fd_h_l': [],
'batched Vp_h_m': [],
'batched P_h_m': [],
'batched timeframes': [],
'batched Ir_h_l_m': [],
'batched Ia_h_l_m': [],
'batched Va_hm': []
"""

for i in range(2):
    for h in range(len(batch_vals["batched H"][i])):
        for l in range(len(L)):
            """if np.isnan(batch_vals["batched R_h_l"][6][h,l]):
                    print(f" R_h_l is nan for Hour: {h}, Meter: {l}")
            if np.isnan(batch_vals["batched Fu_h_l"][6][h,l]):
                print(f" Fu_h_l is nan for Hour: {h}, Meter: {l}")
            if np.isnan(batch_vals["batched Fd_h_l"][6][h,l]):
                print(f"Fd_h_l is nan for Hour: {h}, Meter: {l}")"""
            for m in range(12,22):
                """if np.isnan(batch_vals["batched Vp_h_m"][6][h,m]):
                    print(f" Vp_h_m is nan for Hour: {h}, Meter: {l}, Market: {m}")
                if np.isnan(batch_vals["batched P_h_m"][6][h,m]):
                    print(f" P_h_m is nan for Hour: {h}, Meter: {l}, Market: {m}")
                if np.isnan(batch_vals["batched Va_hm"][6][h,m]):
                    print(f" Va_hm is nan for Hour: {h}, Meter: {l}, Market: {m}")"""
                    
            
                if np.isnan(batch_vals["batched Ia_h_l_m"][i][h,l,m]):
                    print(f" Ia_hlm is nan for Hour: {h}, Meter: {l}, Market: {m}")
                if np.isinf(batch_vals["batched Ia_h_l_m"][i][h,l,m]):
                    print(f" Ia_hlm is inf for Hour: {h}, Meter: {l}, Market: {m}")
                if np.isnan(batch_vals["batched Ir_h_l_m"][i][h,l,m]):
                    print(f" Ir_hlm is nan for Hour: {h}, Meter: {l}, Market: {m}")
                if np.isinf(batch_vals["batched Ir_h_l_m"][i][h,l,m]):
                    print(f" Ir_hlm is inf for Hour: {h}, Meter: {l}, Market: {m}")
                    
    

In [50]:
def get_market_count_dict(x : dict, H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], Fu_h_l : np.array, Fd_h_l : np.array):
    """ function to get a dictionary of the results of the optimization problem.
        The solution is represented as a dataframe for each hour which tells how many assets and how much flex volume is connected to each market for each hour.

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour

    Returns:
        dict: the solution of the optimization problem
    """
    data = []

    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    # Calculate flex volume for this asset, market, and hour
        
                    data.append([hour, load.meter_id, market.name])

    df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
    market_names = [m.name for m in M]
    market_count_dict = {}
    for h, hour in enumerate(H):
        hour_df = df.loc[(df["Hour"] == hour)]
        # Aggregate data by market and hour, counting assets and summing flex volumes
        market_count = hour_df.groupby(["Market", "Hour"]).agg({"Asset Meter ID": "count"}).reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
        flex_volumes = []
        for market_name in market_count["Market"]:
            m = market_names.index(market_name)
            market = M[m]
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X *Fu_h_l[h,l] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * Fd_h_l[h,l] for l, load in enumerate(L) if load.direction != "up")
            else:
                total_flex_volume = 0
                for l, load in enumerate(L):
                    if load.direction == "up":
                        total_flex_volume += x[h, l, m].X * Fu_h_l[h,l]
                    elif load.direction == "down":
                        total_flex_volume += x[h, l, m].X * Fd_h_l[h,l]
                    else:
                        total_flex_volume += min(x[h, l, m].X * Fu_h_l[h,l], x[h, l, m].X * Fd_h_l[h,l])
                
            flex_volumes.append(total_flex_volume)
        market_count["Total Flex Volume"] = flex_volumes
        market_count_dict[hour] = market_count
    return market_count_dict

In [51]:
def run_batched_optimization_model(L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], H : [pd.Timestamp], F : dict, freq_data :pd.DataFrame, P_h_m : np.array, Vp_h_m : np.array, Vm_m : list, R_m : list, R_h_l : np.array, Fu_h_l : np.array, Fd_h_l : np.array, compatible_list : dict, log_filename : str, model_name : str):
    """ Function to create and run an optimization model for bidding in the reserve markets for a given set of meters and markets. 
    The bidding is for historical data. The model is run in batches to avoid memory issues. Therefore the H list is splitted in to batches of 24 hours. So the model is run for each batch.
    The parameters that has values for h are splitted in to batches of the same length as the H batch to make sure that the values are correct for each batch.

    Args:
        L (list(new_meters.PowerMeter]): set of all meters
        M (list(final_markets.ReserveMarket]): set of all markets
        H (list(pd.Timestamp]): set of all hours
        F (dict): Dictionary to find the activation percentages for each market and hour
        freq_data (pd.DataFrame): dataframe with the frequency data
        P_h_m (np.array): The price for each hour and market
        Vp_h_m (np.array): The volume for each hour and market
        Vm_m (list): Minimum volume for each market
        R_m (list): Response time for each market
        R_h_l (np.array): Response time for each load each hour
        Fu_h_l (np.array): Up flex volume for each load that are compatible with up markets for each hour 
        Fd_h_l (np.array): Down flex volume for each load that are compatible with down markets for each hour
        compatible_list (dict): dict of compatible markets for each asset
        log_filename (str): name of the logfile
        model_name (str): name of the model

    Returns:
        aggregated_results (dict): The results from the batched optimization model which includes the model, the decision variables and the values of the decision variables
            model (gp.Model): The model that was run
            x (dict): The decision variables x[h,l,m] which tells if asset l is connected to market m at hour h
            y (dict): The decision variables y[h,m] which tells if market m has a bid at hour h
            w (dict): The decision variables w[h,m] which tells if market m is activated at hour h
            d (dict): The decision variables d[h,l,m] which tells if asset l is compatible with market m at hour h
    """
    batch_size = 24  # For example, batching by 24 hours
    num_batches = math.ceil(len(H) / batch_size)
    aggregated_results = {
        'models': [],
        'x_values': [],
        'y_values': [],
        'w_values': [],
        'd_values': []
    }
    market_name_dict = {m.name : m for m in M}
    result_dicts = []
    for b in range(num_batches):
        # Determine the subset of hours for this batch
        start_index = b * batch_size
        end_index = min((b + 1) * batch_size, len(H))
        batch_H = H[start_index:end_index]

        # Slice numpy arrays for the current batch
        batch_R_h_l = R_h_l[start_index:end_index, :]
        batch_Fu_h_l = Fu_h_l[start_index:end_index, :]
        batch_Fd_h_l = Fd_h_l[start_index:end_index, :]
        batch_Vp_h_m = Vp_h_m[start_index:end_index, :]
        batch_P_h_m = P_h_m[start_index:end_index, :]
        tf = timeframes.TimeFrame(year = 2023, start_month = 6, end_month = 6, start_day = batch_H[0].day, end_day = batch_H[0].day, start_hour = 0, end_hour = 23)

        # the income
        batch_Ir_hlm, batch_Ia_hlm, batch_Va_hm = utils.get_income_dictionaries(H = batch_H, M = M, L =L, freq_data = freq_data, Fu_h_l = batch_Fu_h_l, Fd_h_l = batch_Fd_h_l, P_h_m = batch_P_h_m, Vp_h_m = batch_Vp_h_m, F =F, markets_dict = market_name_dict, timeframe = tf)
       
        # Run the optimization model for this batch
        model, x, y, w, _ = utils.run_optimization_model(L= L, M= M, H = batch_H,F= F, Ir_hlm= batch_Ir_hlm, Ia_hlm= batch_Ia_hlm, Va_hm= batch_Va_hm, Vp_h_m= batch_Vp_h_m, Vm_m=Vm_m, R_m=R_m, R_h_l=batch_R_h_l, Fu_h_l=batch_Fu_h_l, Fd_h_l=batch_Fd_h_l, compatible_list=compatible_list, log_filename=log_filename, model_name=f"{model_name}_batch_{b}")
        # Store results
        aggregated_results['models'].append(model)
        aggregated_results['x_values'].append(x)
        aggregated_results['y_values'].append(y)
        aggregated_results['w_values'].append(w)
        #aggregated_results['d_values'].append(d)
        #utils.test_solution_validity(x, y, w, batch_Va_hm, L, M, batch_H, F)
        mc_dict = get_market_count_dict(x, batch_H, L, M, batch_Fu_h_l, batch_Fd_h_l)
        result_dicts.append(mc_dict)

    # Process aggregated_results as needed
    return aggregated_results, result_dicts

In [52]:
aggregated_results, res_dicts  = run_batched_optimization_model(L= L , M = M, H = H, F = F,freq_data=freq_data, P_h_m=P_h_m , Vp_h_m =Vp_h_m, Vm_m = Vm_m, R_m = R_m, R_h_l = R_h_l, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, compatible_list = compatible_list, log_filename="two_days_batched.log", model_name= "two_days_batched")

/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


Set parameter LogFile to value "two_days_batched.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threadsThread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6573432 rows, 3260208 columns and 20328977 nonzerosOptimize a model with 6573432 rows, 3260208 columns and 20328977 nonzeros
Model fingerprint: 0xa8ba9e6cModel fingerprint: 0xa8ba9e6c
Model has 146161 quadratic objective termsModel has 146161 quadratic objective terms
Model has 1728 quadratic constraintsModel has 1728 quadratic constraints
Variable types: 0 continuous, 3260208 integer (3260208 binary)Variable types: 0 continuous, 3260208 integer (3260208 binary)
Coefficient statistics:Coefficient statistics:
Matrix range     [1e-06, 2e+03]  Matrix range     [

: 

In [24]:
def dissaggregate_results(aggregated_results : dict):
    daily_vals = []
    x_vals = aggregated_results['x_values']
    y_vals = aggregated_results['y_values']
    w_vals = aggregated_results['w_values']
    for day_nr in range(len(x_vals)):
        daily_vals.append({"x_values": x_vals[day_nr], "y_values": y_vals[day_nr], "w_values": w_vals[day_nr]})
    return daily_vals

In [25]:
daily_vals_list = dissaggregate_results(aggregated_results)

In [28]:
len(daily_vals_list[0]["x_values"])

3257232

In [30]:
len(H)* len(L)*len(M)/2

3257232.0

In [33]:

batch_size = 24
dfs = []
for day in range(2):
    #df = pd.DataFrame(columns=["Market", "Hour", "Asset Count","Total Flex Volume [MWh]", "Total capacity revenue [EUR]", "Total activation revenue [EUR]"])
    df = pd.DataFrame(columns=["Market", "Hour", "Asset Count","Total Flex Volume [MWh]"])
    start_index = day * batch_size
    end_index = min((day + 1) * batch_size, len(H))
    batch_H = H[start_index:end_index]
    batch_Fu_h_l = Fu_h_l[start_index:end_index, :]
    batch_Fd_h_l = Fd_h_l[start_index:end_index, :]
    for h, hour in enumerate(batch_H):
        for m, market in enumerate(M):
            if daily_vals_list[day]["y_values"][h,m].X> 0:
                amount_of_assets = sum( daily_vals_list[day]["x_values"][h,l,m].X for l in range(len(L)))
                #capacity_income = sum(daily_vals_list[day]["x_values"][h,l,m].X * Ir_hlm[h, l, m] for l in range(len(L)))
                #activation_income = sum(daily_vals_list[day]["x_values"][h,l,m].X* Ia_hlm[h, l, m] * daily_vals_list[day]["w_values"][h,m].X for l in range(len(L)))
                if market.direction == "up":
                    total_flex_volume = sum(daily_vals_list[day]["x_values"][h,l,m].X * batch_Fu_h_l[h,l] for l in range(len(L)))
                elif market.direction == "down":
                    total_flex_volume = sum(daily_vals_list[day]["x_values"][h,l,m].X* batch_Fd_h_l[h,l] for l in range(len(L)))
                else:
                    total_flex_volume = 0
                    for l, load in enumerate(L):
                        if load.direction == "up":
                            total_flex_volume += daily_vals_list[day]["x_values"][h,l,m].X* batch_Fu_h_l[h,l]
                        elif load.direction == "down":
                            total_flex_volume += daily_vals_list[day]["x_values"][h,l,m].X * batch_Fd_h_l[h,l]
                        else:
                            total_flex_volume += min(daily_vals_list[day]["x_values"][h,l,m].X * batch_Fu_h_l[h,l], daily_vals_list[day]["x_values"][h,l,m].X * batch_Fd_h_l[h,l])
                        
                #df.loc[len(df)] = [market.name, hour, amount_of_assets, total_flex_volume, capacity_income, activation_income]
                df.loc[len(df)] = [market.name, hour, amount_of_assets, total_flex_volume]
    dfs.append(df)



In [34]:
dfs[0]

,Market,Hour,Asset Count,Total Flex Volume [MWh]
0,aFRR down_NO5,2023-06-25 00:00:00+02:00,501.0,2.358485
1,FCR_N_D_1_NO5,2023-06-25 01:00:00+02:00,282.0,1.401498
2,FCR_N_D_1_NO5,2023-06-25 02:00:00+02:00,301.0,1.365446
3,FCR_N_D_1_NO5,2023-06-25 03:00:00+02:00,285.0,1.251855
4,aFRR down_NO5,2023-06-25 05:00:00+02:00,506.0,1.624583
5,aFRR down_NO5,2023-06-25 06:00:00+02:00,510.0,1.881261
6,aFRR down_NO5,2023-06-25 07:00:00+02:00,478.0,1.639083
7,FCR_N_D_1_NO5,2023-06-25 08:00:00+02:00,281.0,1.206908
8,FCR_N_D_1_NO5,2023-06-25 09:00:00+02:00,278.0,1.540171
9,FCR_N_D_1_NO5,2023-06-25 10:00:00+02:00,296.0,1.727881


In [35]:
model, x, y, w, d = utils.run_optimization_model(L= L, M= M, H = H,F= F, Ir_hlm= Ir_hlm, Ia_hlm= Ia_hlm, Va_hm= Va_hm, Vp_h_m= Vp_h_m, Vm_m=Vm_m, R_m=R_m, R_h_l=R_h_l, Fu_h_l=Fu_h_l, Fd_h_l=Fd_h_l, compatible_list=compatible_list, log_filename="two_days.log", model_name="two_days")

Set parameter LogFile to value "two_days.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threadsThread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13146864 rows, 6520416 columns and 40656137 nonzerosOptimize a model with 13146864 rows, 6520416 columns and 40656137 nonzeros
Model fingerprint: 0x5872d303Model fingerprint: 0x5872d303
Model has 232283 quadratic objective termsModel has 232283 quadratic objective terms
Model has 3456 quadratic constraintsModel has 3456 quadratic constraints
Variable types: 0 continuous, 6520416 integer (6520416 binary)Variable types: 0 continuous, 6520416 integer (6520416 binary)
Coefficient statistics:Coefficient statistics:
Matrix range     [1e-06, 2e+03]  Matrix range     [1e-06,

In [36]:
df = pd.DataFrame(columns=["Market", "Hour", "Asset Count","Total Flex Volume [MWh]"])
for h, hour in enumerate(H):
    for m, market in enumerate(M):
        if y[h, m].X > 0:
            amount_of_assets = sum(x[h, l, m].X for l in range(len(L)))
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * Fu_h_l[h,l] for l in range(len(L)))
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * Fd_h_l[h,l] for l in range(len(L)))
            else:
                total_flex_volume = 0
                for l, load in enumerate(L):
                    if load.direction == "up":
                        total_flex_volume += x[h, l, m].X * Fu_h_l[h,l]
                    elif load.direction == "down":
                        total_flex_volume += x[h, l, m].X * Fd_h_l[h,l]
                    else:
                        total_flex_volume += min(x[h, l, m].X * Fu_h_l[h,l], x[h, l, m].X * Fd_h_l[h,l])
                        
            df.loc[len(df)] = [market.name, hour, amount_of_assets, total_flex_volume]


In [37]:
df

,Market,Hour,Asset Count,Total Flex Volume [MWh]
0,aFRR down_NO5,2023-06-25 00:00:00+02:00,501.0,2.358485
1,FCR_N_D_1_NO5,2023-06-25 01:00:00+02:00,282.0,1.401498
2,FCR_N_D_1_NO5,2023-06-25 02:00:00+02:00,301.0,1.365446
3,FCR_N_D_1_NO5,2023-06-25 03:00:00+02:00,285.0,1.251855
4,aFRR down_NO5,2023-06-25 05:00:00+02:00,506.0,1.624583
...,...,...,...,...
108,FCR_N_D_1_NO5,2023-06-26 21:00:00+02:00,245.0,1.259441
109,aFRR down_NO5,2023-06-26 21:00:00+02:00,333.0,1.431239
110,FCR_N_D_1_NO1,2023-06-26 22:00:00+02:00,253.0,1.172821
111,aFRR down_NO5,2023-06-26 22:00:00+02:00,572.0,2.284753


In [39]:
df.loc[df["Hour"] < H[24]] == dfs[0]

,Market,Hour,Asset Count,Total Flex Volume [MWh]
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
5,True,True,True,True
6,True,True,True,True
7,True,True,True,True
8,True,True,True,True
9,True,True,True,True


In [47]:
df.loc[df["Hour"] >= H[24]].reset_index(drop=True) 

,Market,Hour,Asset Count,Total Flex Volume [MWh]
0,aFRR down_NO5,2023-06-26 00:00:00+02:00,513.0,3.585945
1,FCR_N_D_1_NO5,2023-06-26 01:00:00+02:00,312.0,2.553063
2,FCR_N_D_1_NO5,2023-06-26 02:00:00+02:00,308.0,2.362834
3,FCR_N_D_1_NO5,2023-06-26 03:00:00+02:00,280.0,2.286405
4,FCR_N_D_1_NO5,2023-06-26 04:00:00+02:00,291.0,2.138933
5,FCR_N_D_1_NO5,2023-06-26 05:00:00+02:00,294.0,2.144209
6,FCR_N_D_1_NO5,2023-06-26 06:00:00+02:00,263.0,1.087126
7,aFRR down_NO5,2023-06-26 06:00:00+02:00,272.0,1.128576
8,FCR_N_D_2_NO5,2023-06-26 07:00:00+02:00,280.0,1.220036
9,aFRR up_NO5,2023-06-26 07:00:00+02:00,222.0,1.210749


In [46]:
dfs[1]

,Market,Hour,Asset Count,Total Flex Volume [MWh]
0,FCR_N_D_1_NO5,2023-06-26 00:00:00+02:00,161.0,1.083177
1,aFRR down_NO5,2023-06-26 00:00:00+02:00,377.0,2.565478
2,FCR_N_D_1_NO5,2023-06-26 01:00:00+02:00,312.0,2.553063
3,FCR_N_D_1_NO5,2023-06-26 02:00:00+02:00,308.0,2.362834
4,FCR_N_D_1_NO5,2023-06-26 03:00:00+02:00,280.0,2.286405
5,FCR_N_D_1_NO5,2023-06-26 04:00:00+02:00,291.0,2.138933
6,FCR_N_D_1_NO5,2023-06-26 05:00:00+02:00,294.0,2.144209
7,FCR_N_D_1_NO5,2023-06-26 06:00:00+02:00,254.0,1.032849
8,aFRR down_NO5,2023-06-26 06:00:00+02:00,281.0,1.182853
9,FCR_N_D_2_NO5,2023-06-26 07:00:00+02:00,280.0,1.220036


In [ ]:
H_batches = [H[:24], H[24:]]
for index, daily_val in enumerate(daily_vals_list):
    utils.test_solution_validity(daily_val["x_values"], daily_val["y_values"], daily_val["w_values"], Va_hm, L, M, H_batches[index], dominant_directions= dominant_directions, F = F)
    print(f" Solution for day {daily_vals_list.index(daily_val)} is valid")
    # must split the H list into days

### possible reasons for failure of the second day batched version:
# the indexes becomes wrong when the data is batched


In [ ]:
afrr_activation_up = utils.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Up") #  a dataframe of the activation volumes for afrr up for each hour in the timeframe
afrr_activation_down = utils.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Down") #  a dataframe of the activation volumes for afrr down for each hour in the timeframe
frequency_quarter_dict = utils.find_frequency_quarters(freq_df = freq_data, hours = H, index = True) # a dictionary of the frequency quarters for each hour

In [ ]:
afrr_activation_up.isna()

In [ ]:
#sum(x[7, l, 21].X for l in range(len(L)))

In [ ]:
#len(L) * y[7, 21].X

In [ ]:
#round(sum(x[7, 839, m].X for m in range(len(M))), 6)

In [ ]:
import pickle

In [ ]:
# Extract binary variable values from the original model
#new_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}


In [ ]:
"""# Load the saved values
with open('current_x_values_for_week.pkl', 'rb') as f:
    original_x_values = pickle.load(f)

old_dict = utils.get_market_count_dict(original_x_values)
new_dict = utils.get_market_count_dict(new_x_values)

differences = {}
for key in old_dict:
    if not old_dict[key].equals(new_dict[key]):
        differences[key] = (new_dict[key], old_dict[key])
        

for key, (orig_val, mod_val) in differences.items():
    print(f"Difference for hour {key}: \n Original={display(orig_val)}, \n  Modified={display(mod_val)}")"""


In [ ]:
"""# Extract binary variable values from the original model
current_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(current_x_values, f)"""
                